# Rates

## Onderhoudsfunctie

Lees data in en voeg een groepering toe van rate names naar 'currency group'.

Het inlezen en groeperen van de data moet een aparte routine worden

In [11]:
import pandas as pd
import json

In [13]:
date_cols = ["rate_dt","input_dt","actual_dt",'start_term_dt','maturity_term_dt']
keys = ['curve_type','rate_name','rate_type','ccy','time_band']

df = pd.read_csv(r'data\rates_data.csv', 
                 parse_dates = date_cols, 
                 dtype ={"int_basis":'S10', "exchange_name": 'S10'})

In [14]:
# Selectie van range van de laatste 20 dagen

df_20d = df[df.rate_dt >= df['rate_dt'].unique()[-20]].copy()

# Toevoegen van gemiddelde en standaard afwijking - daarna berekenen van z-score

df_gr = df_20d.groupby(keys).agg( {'mid' : ['mean', 'std']}) 
df_gr = df_gr.reset_index()

In [15]:
df_gr

curve_type     rate_name   rate_type  ccy time_band        mid          
                                                                mean       std
0     Capital Pr  AT0000A001X2  End of Day  EUR    ACTUAL  103.31625  0.097796
1     Capital Pr  AT0000A0U3T4  End of Day  EUR    ACTUAL  108.25895  0.118876
2     Capital Pr  AT0000A105W3  End of Day  EUR    ACTUAL  107.22450  0.085713
3     Capital Pr  AT0000A185T1  End of Day  EUR    ACTUAL  109.30945  0.091579
4     Capital Pr  AT0000A1FAP5  End of Day  EUR    ACTUAL  109.33150  0.113268
...          ...           ...         ...  ...       ...        ...       ...
2635       Yield  US912810FT08  End of Day  USD    ACTUAL    1.00000  0.000000
2636       Yield  US9128282R06  End of Day  USD    ACTUAL    1.00000  0.000000
2637       Yield  US912828YF19  End of Day  USD    ACTUAL    1.00000  0.000000
2638       Yield  US912828ZC78  End of Day  USD    ACTUAL    1.00000  0.000000
2639       Yield  US912828ZQ64  End of Day  USD    ACTUAL    1.00000  0.000000

[2640 rows x 7 columns]

In [16]:
def ccy_group (currency):
    ''' 
    Map currencies in A- B- and C-currencies 
    
    input:
    currency       - string : 3 digit currency code
    
    Output:
    currency group - string    
    '''
    
    a_ccy = ['EUR','GBP','USD','CHF']
    b_ccy = ['NOK','NZD','SEK','CZK']
    
    if any(currency in x for x in a_ccy):
        return 'A-currency'
    if any(currency in x for x in b_ccy):
        return 'B-currency'
    else:
        return 'C-currency'

# Map currencies in currency groups
df_gr['ccy_group'] = df_gr['ccy'].map(ccy_group)

In [17]:
df_ccylist = df_gr[['ccy_group','ccy']].drop_duplicates().copy()
df_ccylist.columns = ['ccy_group','ccy']

In [18]:
file = r'data\ccy_group.json'
r = df_ccylist.to_json(file)